# This is a categorical scrapping project
The goal for the project is to create a scrapper that could monitor the products informations in ebay.

It will start from a given search and return all the project it had got.
Then becomes a much intellegent scrapper that can cover the entire site and adjusting to changes.

It will use xpath first, then it will switch to image recognition mode for futher scrapping.

In [18]:
xpath_search_input_field = r'//*[(@id = "gh-ac")]'
xpath_search_button = r'//*[(@id = "gh-btn")]'

In [4]:
ebay_url = "https://www.ebay.com"

In [139]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import os
import logging
from parsel import Selector
from pprint import *
import time
from peewee import *
from playhouse.sqlite_ext import SqliteExtDatabase
import datetime
import re
import copy
import pandas as pd
from IPython.display import display, HTML

In [3]:
given_serach_input = "Iphone"

In [13]:
def web_driver_init():
    driver = webdriver.Chrome()
    driver.get(ebay_url)
    return driver

driver = web_driver_init()

In [19]:
def search_input_insertation(driver):
    try:
        driver.find_element_by_xpath(xpath_search_input_field).send_keys(given_serach_input)
        driver.find_element_by_xpath(xpath_search_button).click()
    except:
        logging.error("Please switch to a better network condition and try again.")

search_input_insertation(driver)

In [124]:
db = SqliteDatabase("ebay_search_spider_2.db")
db.connect()
db.drop_tables([itemOfSearchResult], safe=True)
db.create_tables([itemOfSearchResult], safe=True)
db.close()

class BaseModel(Model):
    class Meta:
        database = db
        
class itemOfSearchResult(BaseModel):
    product_sponsored = BooleanField()
    
    product_title = CharField(unique=False)
    product_tags = CharField(null=True)
    product_status = CharField(null=True)
    
    product_price_from = CharField(null=True)
    product_price_to = CharField(null=True)
    products_price_off = CharField(null=True)
    
    product_shipping = CharField(null=True)
    product_fast_returns = BooleanField()
    product_estimated_delivery_time = CharField(null=True)
    product_guaranteed_by = CharField(null=True)
    
    product_top_rated_plus = BooleanField()
    product_fast_n_free = BooleanField()
    product_buy_spry = BooleanField()
    
    product_watching_or_sold = CharField(null=True)
    
    products_inventory_infomation = CharField(null=True)
    
    product_image_url = TextField(null=True)
    
    scrapped_time = TimeField()

In [152]:
xpath_product_list = """//*[contains(concat( " ", @class, " " ), concat( " ", "li", " " ))]"""

xpath_product_sponsored = """//*[contains(concat( " ", @class, " " ), concat( " ", "promoted-lv", " " ))]//span"""

xpath_product_title = """string(//*[contains(concat( " ", @class, " " ), concat( " ", "vip", " " ))]//text())"""
xpath_product_tags = """//div[@class="lvsubtitle"][1]/text()"""
xpath_product_status = """//div[@class="lvsubtitle"][2]/text()"""

xpath_product_price_from = """//*[contains(concat( " ", @class, " " ), concat( " ", "prRange", " " ))]/text()[1]"""
xpath_product_price_to = """//*[contains(concat( " ", @class, " " ), concat( " ", "prRange", " " ))]/text()[2]"""
xpath_products_price_off = """//*[contains(concat( " ", @class, " " ), concat( " ", "black", " " ))]/text()"""

xpath_product_shipping = """//*[contains(concat( " ", @class, " " ), concat( " ", "ship", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bfsp", " " ))]/text()"""
xpath_product_fast_returns = """//*[contains(concat( " ", @class, " " ), concat( " ", "lvreturns", " " ))]//*[contains(concat( " ", @class, " " ), concat( " ", "bfsp", " " ))]"""
xpath_product_estimated_delivery_time = """//*[contains(concat( " ", @class, " " ), concat( " ", "fnftxt", " " ))]//b/text()"""
xpath_product_guaranteed_by = """//*[(@id = "item1c58f19dc6")]//b/text()"""

xpath_product_top_rated_plus = """//*[contains(concat( " ", @class, " " ), concat( " ", "iconETRS2", " " ))]"""
xpath_product_fast_n_free = """//*[contains(concat( " ", @class, " " ), concat( " ", "fnf-green", " " ))]"""
xpath_product_buy_spry = """//*[contains(@class, "lvdetails")]//a[contains(@href, "BuySPRY")]"""

xpath_product_watching_or_sold = """//*[contains(concat( " ", @class, " " ), concat( " ", "red", " " ))]/text()"""

xpath_product_inventory_infomation = """//*[contains(concat( " ", @class, " " ), concat( " ", "lvoptions", " " ))]//a/text()"""

xpath_product_image_url = """//*[contains(concat( " ", @class, " " ), concat( " ", "img", " " ))]//img/@src"""


xpath_next_page_button = """//*[contains(concat( " ", @class, " " ), concat( " ", "next", " " ))]"""

In [151]:
def collect_page_data(driver):
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight*4);")
        html = driver.page_source

        product_html_list = Selector(text=html).xpath(xpath_product_list).extract()
        for product_html in product_html_list:
            search_result = itemOfSearchResult()

            search_result.product_sponsored = str(Selector(text=product_html).xpath(xpath_product_sponsored).extract_first()).strip()

            search_result.product_title = str(Selector(text=product_html).xpath(xpath_product_title).extract_first()).strip()
            search_result.product_tags = str(Selector(text=product_html).xpath(xpath_product_tags).extract_first()).strip()
            search_result.product_status = str(Selector(text=product_html).xpath(xpath_product_status).extract_first()).strip()

            search_result.product_price_from = str(Selector(text=product_html).xpath(xpath_product_price_from).extract_first()).strip()
            search_result.product_price_to = str(Selector(text=product_html).xpath(xpath_product_price_to).extract_first()).strip()
            search_result.products_price_off = str(str(Selector(text=product_html).xpath(xpath_products_price_off).extract_first())).strip()

            search_result.product_shipping = str(Selector(text=product_html).xpath(xpath_product_shipping).extract_first()).strip()
            if Selector(text=product_html).xpath(xpath_product_fast_returns).extract_first() is None:
                search_result.product_fast_returns = False
            else: 
                search_result.product_fast_returns = True
            search_result.product_estimated_delivery_time = str(Selector(text=product_html).xpath(xpath_product_estimated_delivery_time).extract_first()).strip()
            search_result.product_guaranteed_by = str(Selector(text=product_html).xpath(xpath_product_guaranteed_by).extract_first()).strip()

            if Selector(text=product_html).xpath(xpath_product_top_rated_plus).extract_first() is None:
                search_result.product_top_rated_plus = False
            else:
                search_result.product_top_rated_plus = True
            if Selector(text=product_html).xpath(xpath_product_fast_n_free).extract_first() is None:
                search_result.product_fast_n_free = False
            else:
                search_result.product_fast_n_free = True
            if Selector(text=product_html).xpath(xpath_product_buy_spry).extract_first() is None:
                search_result.product_buy_spry = False
            else:
                search_result.product_buy_spry = True

            search_result.product_watching_or_sold = str(Selector(text=product_html).xpath(xpath_product_watching_or_sold).extract_first()).strip()

            search_result.product_inventory_infomation = str(Selector(text=product_html).xpath(xpath_product_inventory_infomation).extract_first()).strip()

            search_result.product_image_url = str(Selector(text=product_html).xpath(xpath_product_image_url).extract_first()).strip()

            search_result.scrapped_time = str(datetime.datetime.now().strftime("%I:%M%p on %B %d, %Y"))

            search_result.save()
    except:
        logging.error("data parsing error: you may need to update the script")

In [ ]:
def get_to_next_page(driver):
    try:
        driver.find_element_by_xpath(xpath_next_page_button).click()
    except:
        raise Exception("All Page are scrapped")
try:
    while True:
        collect_page_data(driver)
        get_to_next_page(driver)
        print("Successfully Get a new page. ")
except:
    print("All pages are scrapped. Script Finished!")

In [ ]:
for result in itemOfSearchResult.select():
    display(result)